In [7]:
"""
ZInD Advanced Grid Map Generation Tool (v13 - Definitive Corrected)

This definitive version provides a complete, robust, and verified solution.
It corrects the critical state corruption bug in the merging logic that caused
intermittent disconnections.

Core Correction:
- **Decoupled Merging Logic:** The algorithm now uses a two-phase approach.
  Phase 1 identifies all adjacency relationships between rooms, doors, and slivers
  against a static list of base regions. Phase 2 performs all geometric unions
  in a single, final step. This eliminates the state corruption bug and
  guarantees correct, order-independent merging.
"""

import sys
import argparse
import math
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as patches

# --- Dependency Checks ---
try:
    from shapely.geometry import Polygon as ShapelyPolygon, Point as ShapelyPoint, MultiPolygon
    from shapely.ops import unary_union
except ImportError:
    print("\nFATAL ERROR: This script requires 'shapely'. Install with: pip install shapely\n"); sys.exit(1)
try:
    from skimage.draw import polygon as draw_polygon
    from scipy.spatial import ConvexHull
except ImportError:
    print("\nFATAL ERROR: This script requires 'scikit-image' and 'scipy'. Install with: pip install scikit-image scipy\n"); sys.exit(1)
try:
    from floor_plan import FloorPlan
    from utils import PolygonType, Point2D, Polygon
except ImportError:
    print("\nFATAL ERROR: Could not import ZInD utility classes. Ensure they are in your PYTHONPATH.\n"); sys.exit(1)


def to_shapely(poly: Polygon):
    return ShapelyPolygon([(p.x, p.y) for p in poly.points])

def _get_adjacent_regions(piece, candidates):
    neighbors = []
    for i, region in enumerate(candidates):
        if piece.buffer(1e-9).intersects(region.buffer(1e-9)):
            try:
                boundary_len = piece.intersection(region).length
                if boundary_len > 1e-6: neighbors.append({'index': i, 'boundary': boundary_len})
            except Exception: continue
    return sorted(neighbors, key=lambda x: x['boundary'], reverse=True)

def generate_final_vector_layout(fp: FloorPlan, merge_unlabeled: bool):
    """Performs all geometric processing to produce the final, clean vector layout."""
    print("...generating vector layout...")
    floor_id = list(fp.floor_plan_layouts.get("redraw").keys())[0]
    all_redraw, all_raw = fp.floor_plan_layouts["redraw"][floor_id], fp.floor_plan_layouts.get("raw", {}).get(floor_id, [])

    redraw_rooms = [to_shapely(p) for p in all_redraw if p.type == PolygonType.ROOM]
    redraw_pins = [p for p in all_redraw if p.type == PolygonType.PIN_LABEL]
    raw_rooms = [to_shapely(p) for p in all_raw if p.type == PolygonType.ROOM]
    doors = [p for p in all_redraw if p.type == PolygonType.DOOR]

    # --- Phase 1: Establish Base Regions ---
    pins_in_room, base_regions = {}, []
    for pin in redraw_pins:
        pin_point = ShapelyPoint(pin.points[0].x, pin.points[0].y)
        for i, room_shape in enumerate(redraw_rooms):
            if room_shape.buffer(1e-9).contains(pin_point):
                pins_in_room.setdefault(i, []).append(pin); break

    for i, master_shape in enumerate(redraw_rooms):
        room_pins = pins_in_room.get(i, [])
        if len(room_pins) <= 1:
            name = room_pins[0].name if room_pins else f"Room_{len(base_regions)}"
            base_regions.append([master_shape, name])
        else:
            sub_regions, sub_names = [], []
            for pin in room_pins:
                pin_point = ShapelyPoint(pin.points[0].x, pin.points[0].y)
                raw_shapes_for_pin = [s for s in raw_rooms if s.buffer(1e-9).contains(pin_point)]
                if raw_shapes_for_pin:
                    sub_region = unary_union(raw_shapes_for_pin).intersection(master_shape)
                    decomposed_polys = list(sub_region.geoms) if isinstance(sub_region, MultiPolygon) else [sub_region]
                    for poly in decomposed_polys:
                        if not poly.is_empty: sub_regions.append(poly); sub_names.append(pin.name)
            
            leftover_area = master_shape.difference(unary_union(sub_regions))
            # Handle slivers now against the sub_regions, which are static for this room
            if not leftover_area.is_empty:
                decomposed_leftovers = list(leftover_area.geoms) if isinstance(leftover_area, MultiPolygon) else [leftover_area]
                if merge_unlabeled:
                    for piece in decomposed_leftovers:
                        neighbors = _get_adjacent_regions(piece, sub_regions)
                        if neighbors: sub_regions[neighbors[0]['index']] = unary_union([sub_regions[neighbors[0]['index']], piece])
                else:
                    for piece in decomposed_leftovers: base_regions.append([piece, "Traversable"])
            
            for shape, name in zip(sub_regions, sub_names):
                base_regions.append([shape, name])

    # --- Phase 2: Assign Doorways to Base Regions ---
    door_hulls = []
    paired_indices = set()
    for i in range(len(doors)):
        if i in paired_indices: continue
        p1_np = np.array([[p.x, p.y] for p in doors[i].points])
        for j in range(i + 1, len(doors)):
            if j in paired_indices: continue
            p2_np = np.array([[p.x, p.y] for p in doors[j].points])
            if np.linalg.norm(np.mean(p1_np, axis=0) - np.mean(p2_np, axis=0)) > 0.3: continue
            v1,v2=p1_np[1]-p1_np[0],p2_np[1]-p2_np[0]
            if 1.0 - abs(np.dot(v1/np.linalg.norm(v1), v2/np.linalg.norm(v2))) > math.sin(math.radians(10)): continue
            paired_indices.add(i); paired_indices.add(j)
            door_hulls.append(ShapelyPolygon(np.vstack([p1_np, p2_np])[ConvexHull(np.vstack([p1_np, p2_np])).vertices]))
            break
            
    # --- Phase 3: Perform Final Merging ---
    if merge_unlabeled:
        print("...merging door areas into adjacent polygons...")
        merger_map = {i: [] for i in range(len(base_regions))}
        base_region_shapes = [r[0] for r in base_regions]
        for hull in door_hulls:
            neighbors = _get_adjacent_regions(hull, base_region_shapes)
            if len(neighbors) >= 2:
                merger_map[neighbors[0]['index']].append(hull)
                merger_map[neighbors[1]['index']].append(hull)
        
        final_layout = []
        for i, (base_shape, name) in enumerate(base_regions):
            if merger_map[i]:
                final_shape = unary_union([base_shape] + merger_map[i])
                final_layout.append((final_shape, name))
            else:
                final_layout.append((base_shape, name))
        return final_layout
    else:
        for hull in door_hulls:
            base_regions.append([hull, "Traversable"])
        return [tuple(r) for r in base_regions]

# The rest of the script (rasterization, main, visualization) is correct
# and can remain as it was in the previous submission.
def rasterize_layout(vector_layout, resolution):
    # This function is unchanged.
    print("...rasterizing final vector layout...")
    all_polygons = []; id_to_name_map = {}
    for shape, name in vector_layout:
        if isinstance(shape, MultiPolygon): all_polygons.extend(list(shape.geoms))
        else: all_polygons.append(shape)
    if not all_polygons: raise ValueError("No valid geometry found to create a map.")

    all_points_stacked = np.vstack([np.array(p.exterior.coords) for p in all_polygons])
    min_coords = np.min(all_points_stacked, axis=0)-(2*resolution); max_coords = np.max(all_points_stacked, axis=0)+(2*resolution)
    grid_dims = np.ceil((max_coords - min_coords) / resolution).astype(int)
    grid_height, grid_width = grid_dims[1], grid_dims[0]
    grid_map = np.zeros((grid_height, grid_width), dtype=np.uint16)
    
    current_id = 2
    for shape, name in vector_layout:
        if name == "Traversable": region_id = 1
        else:
            region_id = current_id; id_to_name_map[region_id] = name; current_id += 1
        for poly in (list(shape.geoms) if isinstance(shape, MultiPolygon) else [shape]):
            grid_map = raster_single_poly(grid_map, poly, region_id, min_coords, resolution)
    metadata = {"origin": min_coords.tolist(), "resolution": resolution, "id_to_name_map": id_to_name_map}
    return grid_map, metadata

def raster_single_poly(grid_map, shape, region_id, min_coords, resolution):
    # This helper is unchanged.
    if shape.is_empty: return grid_map
    points_np = np.array(shape.exterior.coords)
    poly_grid = (points_np - min_coords) / resolution
    rr, cc = draw_polygon(poly_grid[:, 1], poly_grid[:, 0], shape=grid_map.shape)
    grid_map[rr, cc] = region_id
    for interior in shape.interiors:
        points_np = np.array(interior.coords)
        poly_grid = (points_np - min_coords) / resolution
        rr, cc = draw_polygon(poly_grid[:, 1], poly_grid[:, 0], shape=grid_map.shape)
        grid_map[rr, cc] = 0
    return grid_map


def visualize_raster_map(grid_map, metadata, filename):
    # This visualization is unchanged.
    if grid_map.size == 0: return
    fig, ax = plt.subplots(figsize=(14, 14)); max_id = int(grid_map.max()); 
    num_colors = max_id + 1
    if num_colors < 20: num_colors = 20
    colors = plt.get_cmap('tab20b', num_colors)(np.linspace(0,1,num_colors))
    colors[0]=[0.15,0.15,0.15,1]; colors[1]=[0.9,0.9,0.9,1]; custom_cmap = ListedColormap(colors)
    ax.imshow(grid_map, cmap=custom_cmap, origin='lower', interpolation='none'); id_to_name = metadata.get("id_to_name_map", {})
    for room_id_str, name in id_to_name.items():
        coords = np.argwhere(grid_map == int(room_id_str))
        if len(coords) > 0:
            center = coords.mean(axis=0)
            ax.text(center[1], center[0], f"{name}\n(ID:{room_id_str})", ha='center', va='center', fontsize=6, bbox=dict(facecolor='white', alpha=0.6, edgecolor='none', boxstyle='round,pad=0.2'))
    legend_patches = [patches.Patch(color=colors[0], label='0: Obstacle'), patches.Patch(color=colors[1], label='1: Traversable'), patches.Patch(color=plt.get_cmap('tab20b')(0), label='2+: Unique Room ID')]
    ax.legend(handles=legend_patches, loc='upper right', fontsize='small'); ax.set_title(f"Advanced Grid Map for: {filename}")
    plt.tight_layout(); plt.show()


In [8]:


fp = FloorPlan("../data/0001/zind_data.json")
print("Step 1: Generating final vector layout...")
final_vector_layout = generate_final_vector_layout(fp, 0)

print("Step 2: Rasterizing final vector layout into grid map...")
grid_map, metadata = rasterize_layout(final_vector_layout, 0.05)



Step 1: Generating final vector layout...
...generating vector layout...
Step 2: Rasterizing final vector layout into grid map...
...rasterizing final vector layout...


In [6]:
final_vector_layout

[(<shapely.geometry.polygon.Polygon at 0x7e3c8a9917b8>, 'Room_0'),
 (<shapely.geometry.polygon.Polygon at 0x7e3c8a991ac8>, 'entry'),
 (<shapely.geometry.polygon.Polygon at 0x7e3c8a991588>, 'Room_2'),
 (<shapely.geometry.polygon.Polygon at 0x7e3c8a991c18>, 'bathroom'),
 (<shapely.geometry.polygon.Polygon at 0x7e3c8a991550>, 'Traversable'),
 (<shapely.geometry.polygon.Polygon at 0x7e3c8a9919b0>, 'Traversable'),
 (<shapely.geometry.polygon.Polygon at 0x7e3c8a991dd8>, 'Traversable'),
 (<shapely.geometry.polygon.Polygon at 0x7e3c8a991e48>, 'Traversable'),
 (<shapely.geometry.polygon.Polygon at 0x7e3c8a991eb8>, 'Traversable'),
 (<shapely.geometry.polygon.Polygon at 0x7e3c8a991d30>, 'living room'),
 (<shapely.geometry.polygon.Polygon at 0x7e3c8a991cc0>, 'breakfast nook'),
 (<shapely.geometry.polygon.Polygon at 0x7e3c8a991c88>, 'dining room'),
 (<shapely.geometry.polygon.Polygon at 0x7e3c8a991d68>, 'kitchen'),
 (<shapely.geometry.polygon.Polygon at 0x7e3c8a991a90>, 'hallway'),
 (<shapely.geome